In [31]:
import pandas as pd
import numpy as np

import nltk
# nltk.download('stopwords')
# nltk.download('punkt_tab')

from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import json


# Data Modeling 
# pick two topics to compare
# develop a way to identify which topic a post is talking about 
# check occurences of different topics of conversation over time
# could these different conversation topics appear during a game period more often?


In [29]:


ps = PorterStemmer()

def text2tokens(text):
	stop_words = set(stopwords.words('english'))
	text = str(text)
	text = text.lower()
	textList = word_tokenize(text)
	textList = [word for word in textList if word not in stop_words and len(word) >= 3]

	textList = [ps.stem(word) for word in textList]
	return textList
    
def gen_bow(df, column):
	df['tokens'] = df[column].apply(text2tokens)
	dct = Dictionary(df['tokens'])
	dct.filter_extremes(no_below=5, no_above=0.5)
	df['bow'] = df['tokens'].apply(dct.doc2bow)
	worddict = {}

	for i in range(len(df)):
		for token in df.at[i,'bow']:
			if token in list(worddict.keys()):
				worddict[token] += 1
			else: 
				worddict[token] = 1

	df.drop('tokens', axis=1, inplace=True)	
	return df, worddict



In [ ]:
gdf = pd.read_csv('hw2_step1_games.csv')
rdf = pd.read_csv('hw2_step1_reddit_posts.csv')
posNegWords = pd.read_excel('posNegList.xlsx').dropna()
# print(posNegWords.dtypes)

posWords = [ps.stem(word) for word in posNegWords['Positive Sense Word List'].to_list()]
negWords = [ps.stem(word) for word in posNegWords['Negative Sense Word List'].to_list()]



def get_sentiment(list):
	sentiment = 0
	
	for word in list:
		if word in posWords:
			sentiment += 1
		elif word in negWords:
			sentiment -= 1
	
	return sentiment
			


rdf, word_dict = gen_bow(rdf, 'text')
rdf['tokens'] = rdf['text'].apply(text2tokens)
# rdf['sentiment'] = rdf['tokens'].apply(get_sentiment)

sent_array = np.array(rdf['tokens'].apply(get_sentiment))
max_sent = sent_array.max()
min_sent = sent_array.min()

rdf['sentiment'] = (sent_array - min_sent) / (max_sent - min_sent)

display(rdf.sort_values(by='sentiment'))


,text,time_of_post,game,team,bow,tokens,sentiment
3183,SHIT PISS AND CUM SHIT PISS AND CUM SHIT PISS ...,2019-10-20,[],BOS,"[(53, 69), (476, 69)]","[shit, piss, cum, shit, piss, cum, shit, piss,...",0.000000
3166,"Congrats, Houston!\n\nFUCK THE YANKEES\n\nFUCK...",2019-10-20,[],BOS,"[(22, 11), (82, 1), (248, 11), (320, 1), (369,...","[congrat, houston, fuck, yanke, fuck, yanke, f...",0.152416
1487,FUCK CHRIS SALE AND HIS GOOFY ASS DELIVERY\n\n...,2019-09-10,['401076896'],NYY,"[(12, 1), (22, 8), (34, 2), (58, 1), (81, 1), ...","[fuck, chri, sale, goofi, ass, deliveri, fuck,...",0.226766
912,"Fuck you Kluber, Fuck you Bauer, Fuck you Jose...",2017-10-11,[],NYY,"[(17, 1), (22, 5), (166, 1), (240, 1), (660, 1...","[fuck, kluber, fuck, bauer, fuck, jose, ramire...",0.234201
2459,Fuck this fucking bullshit\n\nDavid price has ...,2018-09-19,['380919110'],NYY,"[(8, 1), (22, 6), (37, 1), (68, 1), (176, 1), ...","[fuck, fuck, bullshit, david, price, domin, ya...",0.234201
...,...,...,...,...,...,...,...
2742,David Ortiz changed my life and I'll always ca...,2016-10-12,[],BOS,"[(2, 2), (3, 7), (13, 2), (14, 4), (15, 2), (1...","[david, ortiz, chang, life, 'll, alway, carri,...",0.639405
5946,Is...this post in the right place? Have I mis...,2016-04-26,['360426114'],TOR,"[(8, 1), (9, 3), (13, 1), (14, 3), (20, 5), (2...","[..., post, right, place, miss, someth, know, ...",0.639405
2478,Gary Sanchez has had one of the most up and do...,2021-06-24,['401228171'],NYY,"[(0, 4), (2, 5), (3, 7), (9, 4), (14, 1), (19,...","[gari, sanchez, one, career, athlet, last, thi...",0.776952
3291,"Excuse the throwaway (for anonymity), but I pr...",2018-10-19,['381018118'],BOS,"[(0, 2), (2, 1), (3, 7), (9, 8), (14, 11), (15...","[excus, throwaway, anonym, promis, say, true, ...",0.914498


In [ ]:
bal_games = gdf.loc[(gdf['home'] == 'BAL') | (gdf['away'] == 'BAL')]
nyy_games = gdf.loc[(gdf['home'] == 'NYY') | (gdf['away'] == 'NYY')]
bos_games = gdf.loc[(gdf['home'] == 'BOS') | (gdf['away'] == 'BOS')]
tb_games = gdf.loc[(gdf['home'] == 'TB') | (gdf['away'] == 'TB')]
tor_games = gdf.loc[(gdf['home'] == 'TOR') | (gdf['away'] == 'TOR')]


def game_outcome(game_ids, team):
    win_diff = 0
    games = []
    for idx, row in gdf.iterrows():
        if row['game'] in game_ids:
            games.append(row)
    for row in games:
        score_diff = row['home-score'] - row['away-score']
        if row['away'] == team:
            score_diff *= -1
        if score_diff > 0:
            win_diff += 1
        elif score_diff < 0:
            win_diff -= 1
    return win_diff

team_ids = ['BAL', 'NYY', 'BOS', 'TB', 'TOR']

for id in team_ids:
    



# for idx, row in rdf.iterrows():
#     team = row['team']
#     game_ids = list(row['game'])
#     if len(game_ids) == 0:
#         rdf.at[idx, 'team_performance'] = 0
#     else:
#         outcome = game_outcome(game_ids, team)
#         rdf.at[idx, 'team_performance'] = outcome

display(rdf)

IndentationError: expected an indented block after 'for' statement on line 26 (916084214.py, line 39)